In [1]:
import sys
print(sys.executable)


c:\Users\JGuru\miniconda3\envs\familytree-env\python.exe


In [2]:
pip install sqlalchemy graphviz


   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.1 MB 1.7 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.1 MB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 2.9 MB/s eta 0:00:00

   ---------------------------------------- 0/3 [greenlet]
   ---------------------------------------- 0/3 [greenlet]
   ---------------------------------------- 0/3 [greenlet]
   ---------------------------------------- 0/3 [greenlet]
   ---------------------------------------- 0/3 [greenlet]
   ------------- -------------------------- 1/3 [graphviz]
   ------------- -------------------------- 1/3 [graphviz]
   ------------- -------------------------- 1/3 [graphviz]
   ------------- -------------------------- 1/3 [graphviz]
   -------------------------- ---

In [3]:
# Full working standalone family tree script with CLI and visualization (Graphviz)

from sqlalchemy import create_engine, Column, Integer, String, Date, ForeignKey, Text
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
from datetime import datetime, date
from graphviz import Digraph

# --- Database Setup ---
Base = declarative_base()

class Person(Base):
    __tablename__ = 'people'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    date_of_birth = Column(Date)
    date_of_death = Column(Date)
    memos = relationship("Memo", back_populates="person")
    events = relationship("Event", back_populates="person")
    relationships = relationship("Relationship", back_populates="person1", foreign_keys="Relationship.person1_id")

class Relationship(Base):
    __tablename__ = 'relationships'
    id = Column(Integer, primary_key=True)
    person1_id = Column(Integer, ForeignKey('people.id'))
    person2_id = Column(Integer, ForeignKey('people.id'))
    relation_type = Column(String)
    person1 = relationship("Person", foreign_keys=[person1_id], back_populates="relationships")
    person2 = relationship("Person", foreign_keys=[person2_id])

class Event(Base):
    __tablename__ = 'events'
    id = Column(Integer, primary_key=True)
    person_id = Column(Integer, ForeignKey('people.id'))
    event_type = Column(String)
    event_date = Column(Date)
    description = Column(Text)
    person = relationship("Person", back_populates="events")

class Memo(Base):
    __tablename__ = 'memos'
    id = Column(Integer, primary_key=True)
    person_id = Column(Integer, ForeignKey('people.id'))
    note = Column(Text)
    person = relationship("Person", back_populates="memos")

engine = create_engine('sqlite:///family_tree.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

# --- Functional Interface ---

def add_person(name, dob_str=None, dod_str=None):
    dob = datetime.strptime(dob_str, "%Y-%m-%d").date() if dob_str else None
    dod = datetime.strptime(dod_str, "%Y-%m-%d").date() if dod_str else None
    person = Person(name=name, date_of_birth=dob, date_of_death=dod)
    session.add(person)
    session.commit()
    print(f"Added person: {person.name} with ID {person.id}")
    return person.id

def add_relationship(p1_id, p2_id, relation_type):
    rel = Relationship(person1_id=p1_id, person2_id=p2_id, relation_type=relation_type)
    session.add(rel)
    session.commit()
    print("Relationship added.")

def add_event(person_id, event_type, event_date_str, description=""):
    event_date = datetime.strptime(event_date_str, "%Y-%m-%d").date()
    evt = Event(person_id=person_id, event_type=event_type, event_date=event_date, description=description)
    session.add(evt)
    session.commit()
    print("Event added.")

def add_memo(person_id, note):
    memo = Memo(person_id=person_id, note=note)
    session.add(memo)
    session.commit()
    print("Memo added.")

def search_person_by_name(name):
    results = session.query(Person).filter(Person.name.ilike(f"%{name}%")).all()
    if results:
        for p in results:
            print(f"ID: {p.id}, Name: {p.name}, DOB: {p.date_of_birth}, DOD: {p.date_of_death}")
    else:
        print("No matching person found.")

def view_person(person_id):
    person = session.get(Person, person_id)
    if not person:
        print("Person not found.")
        return
    print(f"Name: {person.name}")
    print(f"DOB: {person.date_of_birth}, DOD: {person.date_of_death}")
    print("Memos:")
    for m in person.memos:
        print(f"  - {m.note}")
    print("Events:")
    for e in person.events:
        print(f"  - {e.event_type} on {e.event_date}: {e.description}")
    print("Relationships:")
    for r in person.relationships:
        target = session.get(Person, r.person2_id)
        print(f"  - {r.relation_type} → {target.name}")

def visualize_family_tree(root_person_id, depth=2):
    dot = Digraph(comment="Family Tree")
    visited = set()

    def add_node(person_id, current_depth):
        if current_depth > depth or person_id in visited:
            return
        visited.add(person_id)
        person = session.get(Person, person_id)
        if not person:
            return
        label = f"{person.name}\n({person.date_of_birth})"
        dot.node(str(person.id), label)

        relations = session.query(Relationship).filter(Relationship.person1_id == person_id).all()
        for rel in relations:
            target = session.get(Person, rel.person2_id)
            if not target:
                continue
            dot.node(str(target.id), f"{target.name}\n({target.date_of_birth})")
            dot.edge(str(person.id), str(target.id), label=rel.relation_type)
            add_node(target.id, current_depth + 1)

    add_node(root_person_id, 0)
    dot.render("family_tree", format="png", view=True)
    print("Family tree rendered as family_tree.png")

# --- CLI Interface ---

def menu():
    while True:
        print("\n--- Family Tree Menu ---")
        print("1. Add person")
        print("2. Add relationship")
        print("3. Add event")
        print("4. Add memo")
        print("5. Search person")
        print("6. View person details")
        print("7. Visualize family tree")
        print("0. Exit")
        choice = input("Enter your choice: ")

        if choice == "1":
            name = input("Name: ")
            dob = input("Date of birth (YYYY-MM-DD) [optional]: ") or None
            dod = input("Date of death (YYYY-MM-DD) [optional]: ") or None
            add_person(name, dob, dod)

        elif choice == "2":
            p1 = int(input("Person 1 ID: "))
            p2 = int(input("Person 2 ID: "))
            rtype = input("Relation type (e.g., parent, spouse, child): ")
            add_relationship(p1, p2, rtype)

        elif choice == "3":
            pid = int(input("Person ID: "))
            etype = input("Event type: ")
            edate = input("Event date (YYYY-MM-DD): ")
            desc = input("Description: ")
            add_event(pid, etype, edate, desc)

        elif choice == "4":
            pid = int(input("Person ID: "))
            note = input("Memo: ")
            add_memo(pid, note)

        elif choice == "5":
            name = input("Enter name to search: ")
            search_person_by_name(name)

        elif choice == "6":
            pid = int(input("Enter person ID: "))
            view_person(pid)

        elif choice == "7":
            pid = int(input("Enter root person ID to visualize: "))
            depth = int(input("Depth (default 2): ") or 2)
            visualize_family_tree(pid, depth)

        elif choice == "0":
            print("Goodbye!")
            break

        else:
            print("Invalid choice. Try again.")

# Uncomment the line below to run the menu when executing this script
# menu()
